# Query Pattern
* What is total rental cost between 13/03/2014-24/03/2014?
* How much money collected from the car id=2?


# Getting a record by id

In [1]:
c=Customer.objects.get(id=2)
print c

id: 2, Ronald Cunningham


# Getting all records from table Customer

In [2]:
Customer.objects.all()

[<Customer: id: 1, Alexandra Long>, <Customer: id: 2, Ronald Cunningham>, <Customer: id: 3, Jake Torres>, <Customer: id: 4, Woodrow Castillo>, <Customer: id: 5, Rolando Young>, <Customer: id: 6, Erick Gilbert>, <Customer: id: 7, David Castillo>, <Customer: id: 8, Karen Long>, <Customer: id: 9, Erick Gilbert>, <Customer: id: 10, Albert Gilbert>]

In [3]:
# SQL command
print Customer.objects.all().query

SELECT "myapp_customer"."id", "myapp_customer"."first_name", "myapp_customer"."last_name", "myapp_customer"."Address", "myapp_customer"."postcode", "myapp_customer"."telephone", "myapp_customer"."email" FROM "myapp_customer"


# Filter records within range

In [4]:
from datetime import datetime
import pytz
utc=pytz.timezone('UTC')
start_date = utc.localize( datetime.strptime('2014-03-13','%Y-%m-%d') )
stop_date = utc.localize( datetime.strptime('2014-03-24','%Y-%m-%d') )

In [5]:
Rent.objects.filter(rent_date__range=[start_date, stop_date])

[<Rent: id: 3>, <Rent: id: 4>, <Rent: id: 5>, <Rent: id: 6>, <Rent: id: 7>, <Rent: id: 8>, <Rent: id: 9>]

In [6]:
# SQL command
print Rent.objects.filter(rent_date__range=[start_date, stop_date ]).query

SELECT "myapp_rent"."id", "myapp_rent"."rent_date", "myapp_rent"."return_date", "myapp_rent"."cost", "myapp_rent"."car_id", "myapp_rent"."customer_id" FROM "myapp_rent" WHERE "myapp_rent"."rent_date" BETWEEN 2014-03-13 00:00:00 AND 2014-03-24 00:00:00


# Filter less_than_or_equal (__lte)

In [7]:
# rent that happended before or equal 13 March 2014
Rent.objects.filter(rent_date__lte=start_date)

[<Rent: id: 1>, <Rent: id: 2>, <Rent: id: 3>]

In [8]:
# SQL command
print Rent.objects.filter(rent_date__lte=start_date).query

SELECT "myapp_rent"."id", "myapp_rent"."rent_date", "myapp_rent"."return_date", "myapp_rent"."cost", "myapp_rent"."car_id", "myapp_rent"."customer_id" FROM "myapp_rent" WHERE "myapp_rent"."rent_date" <= 2014-03-13 00:00:00


# Filter greater than (__gt)

In [9]:
# rent that happended after 13 March 2014
Rent.objects.filter(rent_date__gt=start_date)

[<Rent: id: 4>, <Rent: id: 5>, <Rent: id: 6>, <Rent: id: 7>, <Rent: id: 8>, <Rent: id: 9>, <Rent: id: 10>, <Rent: id: 11>, <Rent: id: 12>, <Rent: id: 13>, <Rent: id: 14>, <Rent: id: 15>, <Rent: id: 16>, <Rent: id: 17>]

In [10]:
# SQL command
print Rent.objects.filter(rent_date__gt=start_date).query

SELECT "myapp_rent"."id", "myapp_rent"."rent_date", "myapp_rent"."return_date", "myapp_rent"."cost", "myapp_rent"."car_id", "myapp_rent"."customer_id" FROM "myapp_rent" WHERE "myapp_rent"."rent_date" > 2014-03-13 00:00:00


# What is total rental cost between 13/03/2014-24/03/2014?

### Naive solution ( but slow )

In [11]:
%%timeit -n10
total=0
q=Rent.objects.filter(rent_date__range=[start_date, stop_date])
for i in q:
   total=total + i.cost

10 loops, best of 3: 2.33 ms per loop


### Better by Using "aggregration()"

In [12]:
%%timeit -n10
from django.db.models import Sum, Max, Min, Avg
Rent.objects.filter(rent_date__range=[start_date, stop_date]).aggregate(Sum('cost'))

10 loops, best of 3: 879 µs per loop


In [13]:
q=Rent.objects.filter(rent_date__range=[start_date, stop_date])
r=q.aggregate(Sum('cost'))
r

{'cost__sum': Decimal('3309.50')}

In [14]:
Rent.objects.filter(rent_date__range=[start_date, stop_date]).aggregate(Max('cost'))

{'cost__max': Decimal('1310.00')}

# Annotate Count

In [15]:
from django.db.models import Count

In [16]:
q=Car.objects.annotate(Count("rent"))

In [17]:
q[0].rent__count

3

In [18]:
for i in q:
    print "rent__count:%s car:%s"%(i.rent__count, i)

rent__count:3 car:id: 1, Mitsubishi L200
rent__count:3 car:id: 2, Mini Cooper
rent__count:3 car:id: 3, TVR Tuscan
rent__count:3 car:id: 4, BMW Z3
rent__count:2 car:id: 5, Toyota Celica
rent__count:2 car:id: 6, Audi TT
rent__count:1 car:id: 7, Mercedes E320


In [19]:
print Car.objects.annotate(Count("rent")).query

SELECT "myapp_car"."id", "myapp_car"."maker", "myapp_car"."price", "myapp_car"."model", "myapp_car"."year", COUNT("myapp_rent"."id") AS "rent__count" FROM "myapp_car" LEFT OUTER JOIN "myapp_rent" ON ("myapp_car"."id" = "myapp_rent"."car_id") GROUP BY "myapp_car"."id", "myapp_car"."maker", "myapp_car"."price", "myapp_car"."model", "myapp_car"."year"


# Reverse relation

In [20]:
Car.objects.get(id=2)

<Car: id: 2, Mini Cooper>

In [21]:
Car.objects.get(id=2).rent_set.all()

[<Rent: id: 2>, <Rent: id: 4>, <Rent: id: 10>]

In [22]:
# SQL command
print Car.objects.get(id=2).rent_set.all().query

SELECT "myapp_rent"."id", "myapp_rent"."rent_date", "myapp_rent"."return_date", "myapp_rent"."cost", "myapp_rent"."car_id", "myapp_rent"."customer_id" FROM "myapp_rent" WHERE "myapp_rent"."car_id" = 2


# How much money collected from the car id=2?

### Reverse relation (slow)

In [23]:
%%timeit -n1
sum_cost=Car.objects.get(id=2).rent_set.all().aggregate(Sum('cost'))
print sum_cost

{'cost__sum': Decimal('1525.00')}
{'cost__sum': Decimal('1525.00')}
{'cost__sum': Decimal('1525.00')}
1 loop, best of 3: 2.03 ms per loop


In [24]:
print Car.objects.get(id=2).rent_set.all().query

SELECT "myapp_rent"."id", "myapp_rent"."rent_date", "myapp_rent"."return_date", "myapp_rent"."cost", "myapp_rent"."car_id", "myapp_rent"."customer_id" FROM "myapp_rent" WHERE "myapp_rent"."car_id" = 2


### Forward relation

In [25]:
%%timeit -n1
sum_cost=Rent.objects.filter(car__id=2).aggregate(Sum('cost'))
print sum_cost

{'cost__sum': Decimal('1525.00')}
{'cost__sum': Decimal('1525.00')}
{'cost__sum': Decimal('1525.00')}
1 loop, best of 3: 2.27 ms per loop


In [26]:
print Rent.objects.filter(car__id=2).query

SELECT "myapp_rent"."id", "myapp_rent"."rent_date", "myapp_rent"."return_date", "myapp_rent"."cost", "myapp_rent"."car_id", "myapp_rent"."customer_id" FROM "myapp_rent" WHERE "myapp_rent"."car_id" = 2


# Find total income for each car

In [27]:
q=Car.objects.annotate(Sum("rent__cost"))
for i in q:
    print "income:%s car:%s"%(i.rent__cost__sum,i)

income:1529.50 car:id: 1, Mitsubishi L200
income:1525.00 car:id: 2, Mini Cooper
income:2240.00 car:id: 3, TVR Tuscan
income:1119.95 car:id: 4, BMW Z3
income:480.00 car:id: 5, Toyota Celica
income:699.95 car:id: 6, Audi TT
income:514.85 car:id: 7, Mercedes E320


# Q: Why do we need to use revese relation?
# A: Sometimes we need to iterate over all cars to get total cost of each car.

In [28]:
%%timeit -n1
for i in Car.objects.all():
    print "%s\n    %s"%( i, i.rent_set.all().aggregate(Sum('cost')) )

id: 1, Mitsubishi L200
    {'cost__sum': Decimal('1529.50')}
id: 2, Mini Cooper
    {'cost__sum': Decimal('1525.00')}
id: 3, TVR Tuscan
    {'cost__sum': Decimal('2240.00')}
id: 4, BMW Z3
    {'cost__sum': Decimal('1119.95')}
id: 5, Toyota Celica
    {'cost__sum': Decimal('480.00')}
id: 6, Audi TT
    {'cost__sum': Decimal('699.95')}
id: 7, Mercedes E320
    {'cost__sum': Decimal('514.85')}
id: 1, Mitsubishi L200
    {'cost__sum': Decimal('1529.50')}
id: 2, Mini Cooper
    {'cost__sum': Decimal('1525.00')}
id: 3, TVR Tuscan
    {'cost__sum': Decimal('2240.00')}
id: 4, BMW Z3
    {'cost__sum': Decimal('1119.95')}
id: 5, Toyota Celica
    {'cost__sum': Decimal('480.00')}
id: 6, Audi TT
    {'cost__sum': Decimal('699.95')}
id: 7, Mercedes E320
    {'cost__sum': Decimal('514.85')}
id: 1, Mitsubishi L200
    {'cost__sum': Decimal('1529.50')}
id: 2, Mini Cooper
    {'cost__sum': Decimal('1525.00')}
id: 3, TVR Tuscan
    {'cost__sum': Decimal('2240.00')}
id: 4, BMW Z3
    {'cost__sum': Decima

### Better Solution by using "annotation()"

In [29]:
%%timeit -n1
cars=Car.objects.all().annotate(Sum('rent__cost'))
for i in cars:
    print "%s\n    %s"%( i, i.rent__cost__sum ) 

id: 1, Mitsubishi L200
    1529.50
id: 2, Mini Cooper
    1525.00
id: 3, TVR Tuscan
    2240.00
id: 4, BMW Z3
    1119.95
id: 5, Toyota Celica
    480.00
id: 6, Audi TT
    699.95
id: 7, Mercedes E320
    514.85
id: 1, Mitsubishi L200
    1529.50
id: 2, Mini Cooper
    1525.00
id: 3, TVR Tuscan
    2240.00
id: 4, BMW Z3
    1119.95
id: 5, Toyota Celica
    480.00
id: 6, Audi TT
    699.95
id: 7, Mercedes E320
    514.85
id: 1, Mitsubishi L200
    1529.50
id: 2, Mini Cooper
    1525.00
id: 3, TVR Tuscan
    2240.00
id: 4, BMW Z3
    1119.95
id: 5, Toyota Celica
    480.00
id: 6, Audi TT
    699.95
id: 7, Mercedes E320
    514.85
1 loop, best of 3: 6.45 ms per loop


In [30]:
print Car.objects.all().annotate(Sum('rent__cost')).query

SELECT "myapp_car"."id", "myapp_car"."maker", "myapp_car"."price", "myapp_car"."model", "myapp_car"."year", CAST(SUM("myapp_rent"."cost") AS NUMERIC) AS "rent__cost__sum" FROM "myapp_car" LEFT OUTER JOIN "myapp_rent" ON ("myapp_car"."id" = "myapp_rent"."car_id") GROUP BY "myapp_car"."id", "myapp_car"."maker", "myapp_car"."price", "myapp_car"."model", "myapp_car"."year"
